# Genetic Algorithms and Finance
Attempted implementation of https://drive.google.com/file/d/0B9-kA56h5JCMMW4tU25HRExWb1U/view?usp=sharing

## Data Gathering

In [193]:
from getKey import *
import quandl
quandl.ApiConfig.api_key = getKey('quandl.key')

#S&P 500 returns
#SPY data from yahoo finance https://finance.yahoo.com/quote/%5EGSPC/history?period1=-631130400&period2=1544248800&interval=1d&filter=history&frequency=1d
SPY = pd.read_csv('SPY.csv', parse_dates = True, index_col = 0)
SPY["Returns"] = (SPY["Close"] - SPY["Close"].shift(1))/SPY["Close"].shift(1)
data = SPY
data = data.dropna()
print(data.head())

                 Open       High        Low      Close  Adj Close   Volume  \
Date                                                                         
1950-01-04  16.850000  16.850000  16.850000  16.850000  16.850000  1890000   
1950-01-05  16.930000  16.930000  16.930000  16.930000  16.930000  2550000   
1950-01-06  16.980000  16.980000  16.980000  16.980000  16.980000  2010000   
1950-01-09  17.080000  17.080000  17.080000  17.080000  17.080000  2520000   
1950-01-10  17.030001  17.030001  17.030001  17.030001  17.030001  2160000   

             Returns  
Date                  
1950-01-04  0.011405  
1950-01-05  0.004748  
1950-01-06  0.002953  
1950-01-09  0.005889  
1950-01-10 -0.002927  


In [159]:
#True on the index that a crosses over b
def crossover(a, b):
    a.dropna(inplace = True)
    if not isinstance(b, pd.Series):
        b = pd.Series(b, index = a.index)
    else:
        b.dropna(inplace = True)
    minDate = max(a.index[0], b.index[0])
    maxDate = min(a.index[-1], b.index[-1])
    a = a[minDate:maxDate]
    b = b[minDate:maxDate]
    previousPeriodLessThan = a.shift(1) < b.shift(1) #check if in the previous day, a was less than b
    nextPeriodGreaterThan = a > b #in the day after the previous day ("today") a is greater than b
    out = previousPeriodLessThan & nextPeriodGreaterThan
    return out

#True on the index that a crosses under b
def crossunder(a, b):
    a.dropna(inplace = True)
    if not isinstance(b, pd.Series):
        b = pd.Series(b, index = a.index)
    else:
        b.dropna(inplace = True)
    minDate = max(a.index[0], b.index[0])
    maxDate = min(a.index[-1], b.index[-1])
    a = a[minDate:maxDate]
    b = b[minDate:maxDate]
    #note: names not changed, copied from crossover, but still works
    previousPeriodLessThan = a.shift(1) > b.shift(1) #check if in the previous day, a was greater than b
    nextPeriodGreaterThan = a < b #in the day after the previous day ("today") a is less than b
    out = previousPeriodLessThan & nextPeriodGreaterThan
    return out

Indicators: SMA, MACD, Slow Stochastic, RSI, CCI,
Momentum Oscillator, Price Oscillator, Larry Williams, Bollinger Bands and OBV.

In [217]:
import talib
import pandas as pd
close = data.Close
high = data.High
low = data.Low
volume = data.Volume

indicators = []

class Indicator:
    def __init__(self, name, buy, sell):
        self.name = name
        self.buy = buy.dropna()
        self.sell = sell.dropna()
        self.minDate = buy.index[0]
        self.maxDate = buy.index[-1]

In [218]:
#SMA
SMA50 = talib.SMA(close, 50)
SMA200 = talib.SMA(close, 200)
buy = crossover(SMA50, SMA200)
sell = crossunder(SMA50, SMA200)
SMA = Indicator("SMA", buy, sell)
indicators.append(SMA)

In [219]:
#MACD
macd, macdsignal, macdhist = talib.MACD(close, fastperiod=12, slowperiod=26, signalperiod=9)
buy = crossover(macd, macdsignal)
sell = crossunder(macd, macdsignal)
MACD = Indicator("MACD", buy, sell)
indicators.append(MACD)

In [220]:
#RSI
rsi = talib.RSI(close, 14)
buy = crossover(rsi, 30)
sell = crossunder(rsi, 70)
RSI = Indicator("RSI", buy, sell)
indicators.append(RSI)

In [221]:
#CCI
CCI = talib.CCI(high, low, close, timeperiod=14)
buy = crossover(CCI, 100)
sell = crossunder(CCI, -100)
CCI = Indicator("CCI", buy, sell)
indicators.append(CCI)

In [222]:
#Momentum Oscillator
MOM = talib.MOM(close, 9)
buy = crossover(MOM, 100)
sell = crossunder(MOM, 100)
MOM = Indicator("MOM", buy, sell)
indicators.append(MOM)

In [223]:
#Price Oscillator
#Couldn't find

In [224]:
#Williams %R
Williams = talib.WILLR(high, low, close, timeperiod=14)
sell = crossover(Williams, -20)
buy = crossunder(Williams, -80)
Williams = Indicator("Williams %R", buy, sell)
indicators.append(Williams)

In [225]:
#Bollinger Bands
upperband, middleband, lowerband = talib.BBANDS(close, timeperiod=5, nbdevup=2, nbdevdn=2, matype=0)
buy = crossover(middleband, lowerband)
sell = crossunder(middleband, upperband)
bollinger = Indicator("Bollinger Bands", buy, sell)
indicators.append(bollinger)

In [226]:
#OBV
OBV = talib.OBV(close, volume)
OBV_Signal = talib.SMA(OBV, 20)
buy = crossunder(OBV, OBV_signal)
sell = crossover(OB)

In [227]:
for x in indicators:
    print(x.name)

SMA
MACD
RSI
CCI
MOM
Williams %R
Bollinger Bands
